Подключаем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import pickle

Получаем данные о категориях товаров

In [2]:
categories = pd.read_csv('category.csv', names=list(range(0, 7)), sep='[\|\,]', engine='python')
categories.rename(columns={0: categories.iloc[0, 0]}, inplace=True)
categories.drop(0, axis=0, inplace=True)
categories.reset_index(drop=True, inplace=True)

In [3]:
categories.head()

,category_id,1,2,3,4,5,6
0,0,Бытовая электроника,Телефоны,iPhone,None,None,None
1,1,Бытовая электроника,Ноутбуки,None,None,None,None
2,2,Бытовая электроника,Телефоны,Samsung,None,None,None
3,3,Бытовая электроника,Планшеты и электронные книги,Планшеты,None,None,None
4,4,"""Бытовая электроника",Игры,приставки и программы,"Игровые приставки""",None,None


Заполняем пропуски в более низких уровнях иерархии категорий товаров. Удаляем кавычки

In [4]:
for i in range(categories.shape[1]):
    for j in range(categories.shape[0]):
        if categories.iloc[j, i] is None:
            categories.iloc[j, i] = categories.iloc[j, i - 1]
for column in categories:
    categories[column] = categories[column].apply(lambda s: s.replace("\"", ""))

In [5]:
categories.head()

,category_id,1,2,3,4,5,6
0,0,Бытовая электроника,Телефоны,iPhone,iPhone,iPhone,iPhone
1,1,Бытовая электроника,Ноутбуки,Ноутбуки,Ноутбуки,Ноутбуки,Ноутбуки
2,2,Бытовая электроника,Телефоны,Samsung,Samsung,Samsung,Samsung
3,3,Бытовая электроника,Планшеты и электронные книги,Планшеты,Планшеты,Планшеты,Планшеты
4,4,Бытовая электроника,Игры,приставки и программы,Игровые приставки,Игровые приставки,Игровые приставки


Создаем словари для получения по категории 6 уровня категории более высокого уровня

In [6]:
to_labels_lavel = dict()
for i in range(1, 6):
    to_labels_lavel[i] = dict()
for i in range(categories.shape[0]):
    for j in range(1, 6):
        to_labels_lavel[j][int(categories.loc[i, 'category_id'])] = categories.loc[i, j]

Получаем данные об объектах

In [7]:
train = pd.read_csv('train.csv')
print(train.shape)

(489517, 5)


In [12]:
test = pd.read_csv('test.csv')
print(test.shape)

(243166, 4)


In [9]:
train.head()

,item_id,title,description,price,category_id
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1


In [10]:
test.head()

,item_id,title,description,price
0,489517,Стоик журнальный сталь,продам журнальный столик изготавливаю столы из...,10000.0
1,489518,iPhone 5 64Gb,"Телефон в хорошем состоянии. Комплект, гаранти...",12500.0
2,489519,Утеплитель,ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...,250.0
3,489520,Пальто демисезонное,Продам пальто женское (букле) в отличном состо...,1700.0
4,489521,Samsung syncmaster T200N,"Условно рабочий, проблема в панели настройки м...",1000.0


Функция, прводящая слова в тексте к начальной форме и отбрасывающая союзы, междометья, частицы, предикативы и местоимения

In [13]:
def text_normalization(text):
    tokens = nltk.word_tokenize(text)
    tokens = [morph.parse(token)[0].normal_form for token in tokens if (morph.parse(token)[0].tag.POS not in ['INTJ', 'PRCL', 'CONJ', 'PRED', 'NPRO'])]
    new_text =' '.join(tokens)
    return new_text

Соединяем тексты заголовка и объявления в один

In [15]:
for i in range(train.shape[0]):
    train.loc[i, 'words'] = train.loc[i, 'title'] + ' ' + train.loc[i, 'description']
for i in range(test.shape[0]):
    test.loc[i, 'words'] = test.loc[i, 'title'] + ' ' + test.loc[i, 'description']

Преобразуем слова в текстах

In [ ]:
train['words'] = train['words'].apply(text_normalization)
test['words'] = test['words'].apply(text_normalization)

Масштабируем предиктор "цена" и упаковываем в сжатую матрицу

In [14]:
scaler = StandardScaler()
price = scaler.fit_transform(np.array(train['price']).reshape(-1, 1))
price = csr_matrix(price)
test_price = scaler.transform(np.array(test['price']).reshape(-1, 1))
test_price = csr_matrix(test_price)

Приводим слова объявления в нижний регистр и создаем мешок слов из слов объявления и биграмм

In [15]:
count_vect = CountVectorizer(lowercase=True, ngram_range=(1, 2), dtype=np.int16)
bag = count_vect.fit_transform(list(train['words']))
bag = csr_matrix(hstack([bag, price]))
test_bag = count_vect.transform(list(test['words']))
test_bag = csr_matrix(hstack([test_bag, test_price]))

Функция кросс валидации

In [9]:
def cross_val(df, labels):
    
    kf = KFold(n_splits=3)
    accur = []
    it = 1
    for train, test in kf.split(df):
        print('split: ' + str(it))
        it += 1
        # На каждой итерации разбиваем выборку на тестовую и тренировочную
        X_train = df[train]
        X_test = df[test]
        y_train = labels[train]
        y_test = dict()
        y_test[6] = labels[test]
        # Создаем ответы для каждого уроня иерархии категорий
        for i in range(1, 6):
            y_test[i] = []
            for y in y_test[6]:            
                y_test[i].append(to_labels_lavel[i][y])
        # Обучаем модель
        # Предсказываем категорию 6 уровня иерархии
        model = LogisticRegression(C=0.2, multi_class='auto', solver='liblinear', max_iter=200)
        model.fit(X_train, y_train.values.ravel())
        y_pred = dict()
        y_pred[6] = model.predict(X_test)
        # Вычисляем ответы для каждого уровня иерархии категорий
        for i in range(1, 6):
            y_pred[i] = []
            for y in y_pred[6]:
                y_pred[i].append(to_labels_lavel[i][y])
        # Вычисляем accuracy для каждого уровня иерархии категорий        
        new_accur = []
        for i in range(1, 7):
            new_accur.append(accuracy_score(y_test[i], y_pred[i]))
        new_accur = np.array(new_accur)
        accur.append(new_accur)
    # Усредняем accuracy по всем итерациям кросс валидации
    accur = np.array(accur)
    accur = accur.T
    result_accur = []
    for i in range(len(accur)):
        result_accur.append(accur[i].mean())
    return result_accur

Обучаем модель, применяя при этом регуляризацию. 
Предсказываем категории товаров тестовой выборки.
Создаем таблицу с результатами скоринга.
В качестве модели была выбрана Логистическая ререссия, так как она хорошо работает с большим количеством пизнаков.

In [17]:
model = LogisticRegression(C=0.2, multi_class='auto', solver='liblinear')
model.fit(bag, train['category_id'])
test_y_pred_level_6 = model.predict(test_bag)
result = pd.concat([test['item_id'], pd.Series(test_y_pred_level_6, name='category_id')], axis=1)

/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [18]:
result.head()

,item_id,category_id
0,489517,22
1,489518,0
2,489519,15
3,489520,33
4,489521,13


In [19]:
result.to_csv('result.csv')

In [22]:
pickle.dump(bag, open('bag.sav', 'wb'))

In [10]:
bag = pickle.load(open('bag.sav', 'rb'))

При помощи кросс валидации определяем accuracy на каждом уровне иерархии

In [11]:
scores = cross_val(bag, train['category_id'])
for i in range(len(scores)):
    print('Accuracy on ' + str(i + 1) + ' layer: ' + str(round(scores[i], 3)))

split: 1
split: 2
split: 3
Accuracy on 1 layer: 0.959
Accuracy on 2 layer: 0.939
Accuracy on 3 layer: 0.894
Accuracy on 4 layer: 0.893
Accuracy on 5 layer: 0.887
Accuracy on 6 layer: 0.881
